$$\newcommand{\braket}[1]{ \left| { #1 } \right\rangle }$$
$$
\boxed{
\braket{\tilde J(t) }  \equiv \braket{J(t) }  + \braket{\phi(t_0) } \delta (t - {t_0})  \Rightarrow 
  \braket{\phi(t) }  = \int\limits_{ - \infty }^{ + \infty } {d{t_0}\hat G(t-t_0)} \braket{\tilde J(t_0) }  
} 
$$
aa
$$
\frac{\partial }{{\partial z }}{\Psi _n} = {\Psi _{n + 1}} + {\Psi _{n - 1}} - 2{\Psi _n}
$$
Это коммент
$$
 \frac{d}{{dt}}\braket{\phi }  = \hat A \braket{\phi } \Rightarrow \braket{\phi(t) } = \exp( (t-t_0) \hat A) \braket{\phi(t_0) }
$$

$$
\frac{d}{{dt}} \braket{\phi }   = \hat A \braket{\phi }  + \braket{J(t) } \label{qq}
$$

$$
 \hat G(\tau ) = \theta (\tau) \exp (\tau \hat A) 
\Rightarrow \braket{\phi (t) }  = 
\exp( (t-t_0) \hat A)  \braket{\phi(t_0) } 
+\int\limits_{ - \infty }^{ + \infty } {d{t_0}\hat G(t-t_0)}  \braket{J(t_0) }  
$$

$$
 C(z,\bar z )   \frac{d}{{dt}}p(z,\bar z ) 
=   \sigma  \frac{1}{2} {\partial_z }{\partial_{\bar z }}p(z,\bar z ) 
$$

In [1]:
# -*- coding: utf-8 -*-

from NLS.brizers import *
from NLS.nls_adapter import *

from plots.asyn_fig import asyn_figure,sprintf
from plots.k3dsurf import surf,cmps,resize

%matplotlib notebook





A=1-1/8
A=1
#A=0.6
N=16*1024
NZ=1000;
NZ=300;

mm=256+256;
mm=2;
mm=32
mm=2

omagnus=1
#mm=256;
#omagnus=0
flags=-1;

flags=0;
alpha=0*1e-8;

sname='nls-exp-dec';
sname='nls';
#sname="nls-magnus"
#sname='ssf';omagnus=0


caption=sprintf("solver=%s $ord_{magnus}=%d$,flags=%d ",sname.upper(),omagnus,flags)


[pT,pZ]=brizerKM_periods(A);
LT=5*abs(pT);
LZ=8/11*2*2.5*abs(pZ);
NvT=200;
NvZ=300;
tLv=np.linspace(-LT,LT,NvT);
zLv=np.linspace(-LZ*0,LZ,NvZ);
[TTv,ZZv]=np.meshgrid(tLv,zLv);
#figure
#tic


surf(lambda T,Z: np.abs(brizerKM(T,Z,A)),[TTv,ZZv],Xmm=[-1,1],Ymm=[-1,1],axes=['T','z','|\\Psi(T,z)|'],cmp=cmps.twilight_shifted).display()

FFv=brizerKM(TTv,ZZv,A);

surf(FFv.real,[TTv,ZZv],Xmm=[-1,1],Ymm=[-1,1],axes=['T','z','\\Re\\Psi(T,z)']).display()
surf(FFv.imag,[TTv,ZZv],Xmm=[-1,1],Ymm=[-1,1],axes=['T','z','\\Im\\Psi(T,z)']).display()




tt=np.linspace(-LT,LT,N+1);
tt=tt[0:N]
g=1;
[nls,dzu,h]=nls_rescale_to(tt,sname,g);
hz=LZ/NZ; 
dz=hz* dzu/mm

nls.reset(dt=dz,nm=[4,4],omagnus=omagnus,alpha=alpha,w=1/3,flags=flags);




x0=brizerKM(tt,0,A);

nr=np.linalg.norm(x0)*(np.max(tt)-np.min(tt))/N;

z=np.array([0.0,nr,-np.inf]);

xxr=np.zeros((NZ,N), dtype=complex);
xxe=np.zeros((NZ,N), dtype=complex);





def callback(af,i,A,tt,nls,z,hz,mm):           
    
    global caption,xxr,xxe
    
    
    xe=brizerKM(tt,z[0],A);
    if i==0:
        nls.x=xr=xe;
    else:
        xr=nls(rep=int(mm),pp=1);
        
    tcpu,niter=nls.elapsed;
    
    xxr[i,:]=xr;
    xxe[i,:]=xe;
    
    z[0]+=hz;
    
    
    
    # ====   Grapics ====
    
    
    axr=np.abs(xr);
    axe=np.abs(xe);
    
    af[1](1,(tt,axr),color='#77f',legend='$|\\Psi_{num}|$',linewidth=1.5)
    af[1](2,(tt,axe),color='#00f',legend='$|\\Psi_{an}|$',linewidth=0.5)
    
    af[1](3,(tt,np.real(xr)),color='#3f3',legend='$Re\\Psi_{num}$',linewidth=1.5)
    af[1](4,(tt,np.real(xe)),color='#070',legend='$Re\\Psi_{an}$',linewidth=0.5)
    
    af[1].set_title("[%d] %s elapsed: $\\tau_{cpu}$=%3.3f sec iter=%d",i+1,caption,tcpu,niter)
    
    nr=z[1];
    
    err_r=(np.abs(axr-axe)/nr)**2;
    err_f=(np.abs(xr-xe)/nr)**2;
    
    af[2](1,(tt,err_r),color='#33f',legend='$\\delta\\rho$',linewidth=1.5)
    af[2](2,(tt,err_f),color='#3f3',legend='$\\delta\\Psi$',linewidth=1.5)
    
    em=10*np.log10(np.max(err_f));
    if z[2]<em:
        z[2]=em;
    
    af[2].set_title("$\\epsilon=%3.2f [\\epsilon_{max}:%3.2f][dB]$  ",em,z[2]);
    
    af[2].set_ylim((1e-12,100))
    

def oncomplete(xxr,NvZ,NvT,TTv,ZZv):
    
    global caption
    
    print(caption)
    
    xxv=resize(np.abs(xxr),(NvZ,NvT));
    surf(xxv,[TTv,ZZv],Xmm=[-1,1],Ymm=[-1,1],axes=['T','z','|\\Psi(T,z)|'],title=caption,cmp=cmps.twilight_shifted).display()
    
    xxv=resize(xxr.real,(NvZ,NvT));
    surf(xxv,[TTv,ZZv],Xmm=[-1,1],Ymm=[-1,1],axes=['T','z','\\Re\\Psi(T,z)'],title=caption).display()
    
    xxv=resize(xxr.imag,(NvZ,NvT));
    surf(xxv,[TTv,ZZv],Xmm=[-1,1],Ymm=[-1,1],axes=['T','z','\\Im\\Psi(T,z)'],title=caption).display()
    


af=asyn_figure(figsize=(10,8))
af[1].reset(211);
af[2].reset(212, yscale='log');
print('Start....')

af(NZ,lambda af,i: callback(af,i,A,tt,nls,z,hz,mm)).display();
#af(NZ,lambda af,i: callback(af,i,A,tt,nls,z,hz,mm),lambda : oncomplete(xxr,NvZ,NvT,TTv,ZZv) ).display();

    


ModuleNotFoundError: No module named 'NLS'

In [ ]:
oncomplete(xxr,NvZ,NvT,TTv,ZZv)

In [ ]:
oncomplete(xxr,NvZ,NvT,TTv,ZZv)

In [ ]:
fs=asyn_figure(figsize=(10,8))
fs[1].reset(yscale='log');
xxe.shape,xxr.shape
dmi=np.abs(xxe-xxr).min(0);

In [ ]:
efxx=np.abs(xxe-xxr)
errmin=efxx.min(1);
errmax=efxx.max(1);
drxx=np.abs(np.abs(xxe)-np.abs(xxr))
rerrmin=drxx.min(1);
rerrmax=drxx.max(1);

zz=hz*np.arange(NZ);
zmin,zmax=zz.min,zz.max
errmin.shape,errmax.shape,zz.shape


In [ ]:
xxe.shape

In [ ]:

#fs[1].ax.clear()
fs[1].ax.fill_between(zz,errmin, errmax,facecolor='#007',label='err')
fs[1].ax.fill_between(zz,rerrmin, rerrmax,facecolor='#700',label='rer')
fs[1].ax.legend(loc='upper left')
zmin,zmax=zz.min(),zz.max()
fs[1].ax.set_xlim(zmin,zmax)
fs[1].ax.set_ylim(1e-10,1)
fs[1].ax.grid(True,'both')

In [ ]:
zmax

In [6]:

dzu,hz* dzu/mm

(135990.9906674534, 1830.8850066050982)

In [4]:
dz=hz* dzu/mm

In [3]:
from NLS.brizers import *

ModuleNotFoundError: No module named 'NLS'

In [2]:
import os 
os.getcwd()

'V:\\ipc\\py.modules\\Jupyter_scripts'